# Notebook d'entrainement d'un LSTM avec le dataset encodé

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler , OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

from sklearn.compose import ColumnTransformer
from keras.preprocessing.sequence import TimeseriesGenerator


import tensorflow as tf

import scipy.stats as stats

from keras.optimizers import Adam
from keras.layers import Input, LSTM, Dense, RepeatVector,Activation, TimeDistributed
from keras.models import Model

import pickle
import joblib
import mlflow
import mlflow.keras
from mlflow.models.signature import ModelSignature
from mlflow.types import ColSpec, TensorSpec
from mlflow.types import Schema


import datetime
import io

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.regularizers import L2


In [2]:
# nom de fichier et chemin relatif
filename = 'Predict_cleaned_csv.csv'
path = '../../data/metrics/'
# création d'un dataframe à partir du csv de données
df_cleaned = pd.read_csv(path+filename, index_col=0)
df_cleaned.head(2)


,name_modules,name_counters_modules,value_counters_modules,source_events,criticality_events,identification_events,varnishLevelsTargetvolume,varnishLevelsTotalvolume,hour,day,month,time_to_next_event
created_at,,,,,,,,,,,,
2022-04-15 05:55:06.829,Print Engine 1,3D Varnish Counter,1792992,PLC,INFO,391,36192.322612,100000,5,15,4,0.0
2022-04-15 05:55:06.829,iFoil L,Total Pages Counter,22881,PLC,INFO,391,36192.322612,100000,5,15,4,0.0


Verifier la distribution normal des données numeriques

In [3]:
# One-hot encode categorical columns 
cat_columns = ['name_modules', 'name_counters_modules', 'source_events']

# Create the encoder
encoder = ColumnTransformer(
    transformers=[
        ("OneHot", OneHotEncoder(), cat_columns)],
    remainder='passthrough'
)

# Le ColumnTransformer et OneHotEncoder renvoient une matrice sparse pour économiser de l'espace mémoire
# Cependant, cette matrice sparse ne peut pas être convertie directement en un DataFrame pandas
# Nous devons d'abord la convertir en une matrice dense
# Cette conversion est nécessaire pour pouvoir utiliser les données transformées dans notre DataFrame
encoded_columns = encoder.fit_transform(df_cleaned[cat_columns])

# Nous obtenons les noms des nouvelles colonnes créées par l'encodage one-hot
# Ces noms seront utilisés pour créer notre nouveau DataFrame
encoded_cols_names = encoder.get_feature_names_out(input_features=cat_columns)

# Maintenant, nous convertissons la matrice sparse en une matrice dense
# Sans cette conversion, nous ne pourrions pas convertir la matrice en DataFrame
encoded_columns_dense = encoded_columns.toarray()

# Nous créons un nouveau DataFrame avec les colonnes encodées et l'index original
# Il est important de conserver l'index original pour pouvoir fusionner ce nouveau DataFrame avec notre DataFrame original
df_encoded = pd.DataFrame(encoded_columns_dense, columns=encoded_cols_names, index=df_cleaned.index)

# Nous supprimons les colonnes catégorielles originales de notre DataFrame original
# Nous n'avons plus besoin de ces colonnes car nous avons maintenant une version encodée de ces données
df_cleaned.drop(cat_columns, axis=1, inplace=True)

# Nous fusionnons notre DataFrame original et notre DataFrame encodé
# Cela nous donne un DataFrame final qui contient toutes nos données originales, plus les données encodées
df_cleaned = pd.concat([df_cleaned, df_encoded], axis=1)

# Enfin, nous vérifions le résultat pour s'assurer que tout a fonctionné comme prévu
df_cleaned.head()




,value_counters_modules,criticality_events,identification_events,varnishLevelsTargetvolume,varnishLevelsTotalvolume,hour,day,month,time_to_next_event,OneHot__name_modules_Print Engine 1,...,OneHot__source_events_ICB n°6,OneHot__source_events_ICB n°7,OneHot__source_events_ICB n°8,OneHot__source_events_Kernel,OneHot__source_events_PLC,OneHot__source_events_Pilot,OneHot__source_events_RCB n°1,OneHot__source_events_RCB n°2,OneHot__source_events_RCB n°3,OneHot__source_events_iFoil
created_at,,,,,,,,,,,,,,,,,,,,,
2022-04-15 05:55:06.829,1792992,INFO,391,36192.322612,100000,5,15,4,0.000,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2022-04-15 05:55:06.829,22881,INFO,391,36192.322612,100000,5,15,4,0.000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2022-04-15 05:55:06.829,31092,INFO,391,36192.322612,100000,5,15,4,688.575,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2022-04-15 06:06:35.404,1792992,INFO,391,36192.322612,100000,6,15,4,0.000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2022-04-15 06:06:35.404,1792992,INFO,330,36192.322612,100000,6,15,4,0.000,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [4]:
mapping_dict = {'INFO': 0, 'WARNING': 1, 'ERROR': 2}
df_cleaned['criticality_events'] = df_cleaned['criticality_events'].map(lambda x: mapping_dict.get(x, x))

# Check the result
df_cleaned['criticality_events']

created_at
2022-04-15 05:55:06.829    0
2022-04-15 05:55:06.829    0
2022-04-15 05:55:06.829    0
2022-04-15 06:06:35.404    0
2022-04-15 06:06:35.404    0
                          ..
2022-12-12 08:20:17.777    0
2022-12-12 08:20:17.777    0
2022-12-12 08:21:18.076    0
2022-12-12 08:21:18.076    0
2022-12-12 08:21:18.076    0
Name: criticality_events, Length: 244083, dtype: int64

In [5]:
num_columns = ['value_counters_modules', 'identification_events', 'varnishLevelsTargetvolume', 'varnishLevelsTotalvolume','hour','day','month','time_to_next_event']

# Instanciate the scaler
scaler = StandardScaler()

# Fit the scaler and transform the numerical columns
df_cleaned[num_columns] = scaler.fit_transform(df_cleaned[num_columns])

In [6]:
df_cleaned.head()

,value_counters_modules,criticality_events,identification_events,varnishLevelsTargetvolume,varnishLevelsTotalvolume,hour,day,month,time_to_next_event,OneHot__name_modules_Print Engine 1,...,OneHot__source_events_ICB n°6,OneHot__source_events_ICB n°7,OneHot__source_events_ICB n°8,OneHot__source_events_Kernel,OneHot__source_events_PLC,OneHot__source_events_Pilot,OneHot__source_events_RCB n°1,OneHot__source_events_RCB n°2,OneHot__source_events_RCB n°3,OneHot__source_events_iFoil
created_at,,,,,,,,,,,,,,,,,,,,,
2022-04-15 05:55:06.829,1.029711,0,0.312818,-0.869693,0.011087,-1.095119,-0.117225,-1.621751,-0.032645,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2022-04-15 05:55:06.829,-0.718278,0,0.312818,-0.869693,0.011087,-1.095119,-0.117225,-1.621751,-0.032645,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2022-04-15 05:55:06.829,-0.710170,0,0.312818,-0.869693,0.011087,-1.095119,-0.117225,-1.621751,0.226665,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2022-04-15 06:06:35.404,1.029711,0,0.312818,-0.869693,0.011087,-0.925836,-0.117225,-1.621751,-0.032645,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2022-04-15 06:06:35.404,1.029711,0,-0.310676,-0.869693,0.011087,-0.925836,-0.117225,-1.621751,-0.032645,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# Définir les caractéristiques d'entrée et les cibles
X = df_cleaned.drop(['criticality_events', 'time_to_next_event'], axis=1)
y_criticality = df_cleaned['criticality_events']

y_time = df_cleaned['time_to_next_event']

# Diviser les données en ensembles d'entraînement et de test
X_temp, X_test, y_criticality_temp, y_criticality_test, y_time_temp, y_time_test = train_test_split(X, y_criticality, y_time, test_size=0.2, shuffle=False)
# (Nous créons un ensemble de test qui ne sera pas utilisé pendant l'entraînement)

# Diviser les données d'entraînement en ensembles d'entraînement et de validation
X_train, X_val, y_criticality_train, y_criticality_val, y_time_train, y_time_val = train_test_split(X_temp, y_criticality_temp, y_time_temp, test_size=0.12, shuffle=False)


# Dans le premier train_test_split, nous divisons le jeu de données entier (100%) en deux parties : 
# une partie pour l'entraînement (80%) et une partie pour le test (20%). 
# Donc, "X_temp", "y1_temp" et "y2_temp" contiennent 80% des données originales.

# Ensuite, dans le second train_test_split, nous prenons ces 80% des données (X_temp, y1_temp, y2_temp) 
# et nous les divisons à nouveau en deux parties : une partie pour l'entraînement final (88% de 80%, soit  environ 70% des données originales)
# et une partie pour la validation (12% de 80%, soit environ 10% (9.6%) des données originales).

# Cela signifie que, au final, nous avons environ 70% des données pour l'entraînement, 10% pour la validation et 20% pour le test. 




Le batch_size est défini lors de la création de l'objet TimeseriesGenerator parce qu'il affecte la manière dont les données sont préparées pour l'entraînement du modèle.

Dans le contexte de l'apprentissage en profondeur, un "batch" est un sous-ensemble des données d'entraînement qui est utilisé pour mettre à jour les poids du modèle lors d'une itération de l'algorithme d'optimisation (comme la descente de gradient stochastique).

Lorsque vous créez un TimeseriesGenerator, vous spécifiez un batch_size, qui détermine le nombre d'échantillons de séquences temporelles qui seront regroupés dans un seul batch. Ce batch sera ensuite utilisé pour une seule mise à jour des poids du modèle lors de l'entraînement.

Si vous définissiez le batch_size plus tard, lors de l'entraînement du modèle, vous ne pourriez pas contrôler la manière dont vos données sont préparées pour l'entraînement. En définissant le batch_size à l'avance, vous vous assurez que vos données sont préparées de manière optimale pour votre modèle spécifique et votre configuration d'entraînement.

In [8]:
# Définir le nombre de pas de temps
n_input = 25

# Générateurs pour l'entraînement
train_generator_criticality = TimeseriesGenerator(X_train.values, y_criticality_train, length=n_input, batch_size=32)
train_generator_time = TimeseriesGenerator(X_train.values, y_time_train.values, length=n_input, batch_size=32)

# Générateurs pour la validation
val_generator_criticality = TimeseriesGenerator(X_val.values, y_criticality_val, length=n_input, batch_size=32)
val_generator_time = TimeseriesGenerator(X_val.values, y_time_val.values, length=n_input, batch_size=32)

# Générateurs pour le test
test_generator_criticality = TimeseriesGenerator(X_test.values, y_criticality_test, length=n_input, batch_size=32)
test_generator_time = TimeseriesGenerator(X_test.values, y_time_test.values, length=n_input, batch_size=32)

In [9]:
# Nombre de caractéristiques d'entrée
n_features = X_train.shape[1]

# Construction du modèle pour la criticité des événements
model_criticality = Sequential()
# Encodeur
model_criticality.add(LSTM(50, activation='tanh', input_shape=(n_input, n_features)))
model_criticality.add(RepeatVector(50))   # Nous prédisons 50 pas de temps à l'avance

model_criticality.add(LSTM(50, activation='tanh', return_sequences=True))
model_criticality.add(TimeDistributed(Dense(len(np.unique(y_criticality)), activation='softmax')))

# e plus, vous pouvez vouloir changer votre fonction de perte pour une fonction de perte de classification, 
# comme 'categorical_crossentropy' si vos classes sont one-hot encodées ou 'sparse_categorical_crossentropy' si vos classes sont des entiers.
# Compilation du modèle
model_criticality.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')


In [10]:
# Construction du modèle pour le temps jusqu'au prochain événement
model_time_to_event = Sequential()

# Encodeur
model_time_to_event.add(LSTM(50, activation='tanh', input_shape=(n_input, n_features)))
model_time_to_event.add(RepeatVector(50))  # Nous prédisons 50 pas de temps à l'avance

# Décodeur
model_time_to_event.add(LSTM(50, activation='tanh', return_sequences=True))
model_time_to_event.add(TimeDistributed(Dense(1)))  # 1 car nous avons une seule caractéristique en sortie

# Compilation du modèle
model_time_to_event.compile(optimizer='adam', loss='mse', metrics=['mae', 'mape'])

In [11]:
# Définition des callbacks
callbacks_model_criticality = [
    EarlyStopping(patience=5, verbose=1),  
    ModelCheckpoint('model_criticality_checkpoint.h5', verbose=1, monitor='val_loss', mode='min',save_best_only=True, save_weights_only=True)
]


# Définition des callbacks
callbacks_model_time = [
    EarlyStopping(patience=5, verbose=1),  
    ModelCheckpoint('model_time_to_event_checkpoint.h5', verbose=1,monitor='val_loss', mode='min', save_best_only=True, save_weights_only=True)
]

In [12]:
# # Démarrez une nouvelle run MLflow
# with mlflow.start_run(run_name="criticality_model_run"):
#     mlflow.keras.autolog()
#     # Enregistrez les hyperparamètres
#     mlflow.log_param("epochs", 50)
#     mlflow.log_param("batch_size", 32)
#     mlflow.log_param("validation_split", 0.2)
#     mlflow.log_param("early_stopping_patience", 5)
    
    # Entraînez le modèle
history = model_criticality.fit(train_generator_criticality, epochs=1, validation_data=val_generator_criticality, callbacks=callbacks_model_criticality , verbose=1)
    
    # # Enregistrez les métriques
    # for epoch in range(50):
    #     mlflow.log_metric("loss", history.history['loss'][epoch])
    #     mlflow.log_metric("val_loss", history.history['val_loss'][epoch])
    
    # # Enregistrez le modèle
    # mlflow.keras.log_model(model_criticality, "model")

# # Faites de même pour le deuxième modèle
# with mlflow.start_run(run_name="time_to_event_model_run"):
#     # Enregistrez les hyperparamètres
#     mlflow.log_param("epochs", 50)
#     mlflow.log_param("batch_size", 32)
#     mlflow.log_param("validation_split", 0.2)
#     mlflow.log_param("early_stopping_patience", 5)
    
    # Entraînez le modèle
history = model_time_to_event.fit(train_generator_time, epochs=1, validation_data=val_generator_time, callbacks=callbacks_model_time, verbose=1)
    
    # # Enregistrez les métriques
    # for epoch in range(50):
    #     mlflow.log_metric("loss", history.history['loss'][epoch])
    #     mlflow.log_metric("val_loss", history.history['val_loss'][epoch])
    
    # # Enregistrez le modèle
    # mlflow.keras.log_model(model_time_to_event, "model")

InvalidArgumentError: Graph execution error:

Detected at node 'Equal' defined at (most recent call last):
    File "C:\Program Files\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Program Files\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "p:\HubAcademy\AI\MGI\Prediction-Erreur-JetVarnish3D\venv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "p:\HubAcademy\AI\MGI\Prediction-Erreur-JetVarnish3D\venv\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "p:\HubAcademy\AI\MGI\Prediction-Erreur-JetVarnish3D\venv\lib\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "p:\HubAcademy\AI\MGI\Prediction-Erreur-JetVarnish3D\venv\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever(